# Scikit-learn Interface

It is possible to parse scikit-learn models (DecisionTreeClassifier and RandomForestClassifier) into GeFs with the functions provided in `gefs.sklearn_utils`. This notebook is a brief demonstration on how to do that.

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from gefs.sklearn_utils import tree2pc, rf2pc

In [2]:
# Define a synthetic dataset
n_samples = 100
n_features = 20
n_classes = 2

X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=2, n_redundant=2, n_repeated=0, 
                           n_classes=n_classes, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, 
                           hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
# We need to specify the number of categories of each feature (with 1 for continuous features).
ncat = np.ones(n_features+1)  # Here all features are continuous
ncat[-1] = n_classes  # The class variable is naturally categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [3]:
# Train a decision tree
tree = DecisionTreeClassifier().fit(X_train, y_train)

The parsing is done by the `tree2pc` function, which receives the scikit-learn model, the training data, and the number of categories of each variable `ncat`. Other relevant parameters are 
- `learnspn`: The number of samples (at a given leaf) required to run LearnSPN. If set to a high number (say np.Inf), then LearnSPN is never run and all leaves are modelled by fully factorised distributions (all variables are modelled as independent).
- `minstd`: The minimum standard deviation of Gaussian Leaves. This is important for leaves with a small number of datapoints, where the standard deviation is probably understimated. The default value is 1.0 which is 'smooths' the distribution quite aggressively.
- `smoothing`: This is the additive smoothing (Laplace smoothing) applied to categorical variables. This is the reason why, for complete data, GeDTs (GeFs) with fully factorised leaves do not yield the exact same conditional distribution as scikit-learn DTs (RFs), which apply no smoothing. Naturally, one can set it to 0.0 to recover the same conditional distribution.

In [4]:
# Parse it to a Generative Decision Tree (GeDT)
GeDT = tree2pc(tree, X_train, y_train, ncat, learnspn=np.Inf, minstd=.1, smoothing=0.1)

In [5]:
tree.predict_proba(X_test)

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [6]:
pred, prob = GeDT.classify_avg(X_test, return_prob=True)
prob

array([[0.00236967, 0.99763033],
       [0.00236967, 0.99763033],
       [0.00236967, 0.99763033],
       [0.04545455, 0.95454545],
       [0.99744898, 0.00255102],
       [0.99744898, 0.00255102],
       [0.00236967, 0.99763033],
       [0.00236967, 0.99763033],
       [0.99744898, 0.00255102],
       [0.00236967, 0.99763033]])

In [7]:
# If we set smoothing to zero we recover the exact same conditional distribution
GeDT = tree2pc(tree, X_train, y_train, ncat, learnspn=np.Inf, minstd=.1, smoothing=0.0)
pred, prob = GeDT.classify(X_test, return_prob=True)
prob

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

The parsing of Random Forests is done via `rf2pc` which has the same parameters as `tree2pc`.

In [8]:
rf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)

In [9]:
GeF = rf2pc(rf, X_train, y_train, ncat, learnspn=np.Inf, minstd=1., smoothing=1e-6)

In [10]:
pred, prob = GeF.classify_avg(X_test, return_prob=True)
prob

array([[0.29000003, 0.70999997],
       [0.10000002, 0.89999998],
       [0.10000008, 0.89999992],
       [0.36000006, 0.63999994],
       [0.74000001, 0.25999999],
       [0.74000002, 0.25999998],
       [0.32999996, 0.67000004],
       [0.42999992, 0.57000008],
       [0.78999997, 0.21000003],
       [0.23000002, 0.76999998]])

Internally, `rf2pc` reproduces the boostrapping used to create the Random Forest, so provided the pair `X, y` passed to `rf2pc` is the same used to train the Random Forest, the distributions at the leaves will respect the conditional distribution defined by the Random Forest (if `learnspn=np.Inf` and `smoothing=0.0`). This is the most natural use case and the one we assume in the experiments reported in the paper.

Note that it is also possible to pass another pair of `X, y`. However, in this case there is no guarantee that every leaf in the model will have a corresponding set of datapoints. In these 'empty' leaves, the distributions are either uniform with counts given by `smoothing` (categorical variables) or normal with mean zero and standard deviation `minstd` (continuous variables). If you have an use case for this, remember to set `smoothing` to something larger than 0. That is because the weights of sum nodes are estimated (in the maximum likelihood sense) by the number of data points pertaining to each child so that empty leaves will have weight zero and not contribute to the overall distribution defined by the GeF.

## General comments on the API
There are four different functions to do classification with GeFs.
- `classify`
- `classify_avg`
- `classify_lspn`
- `classify_avg_lspn`

The first two, `classify` and `classify_avg`, exploit class factorised leaves to run inference faster (propagate the probabilities of all classes at once). That, of course, only works if the leaves are class factorised (e.g. `learnsp=np.Inf`). Otherwise, one should use `classify_lspn` and `classify_avg_lspn` which work with any PC (in particular those with a LearnSPN network at the leaves, hence the name).

The other important distinction is that `avg` methods assume a model learned as an ensemble and performs inference by 'averaging' the distribution of each of the base models. These are the methods that match the original Random Forest in terms of classification (with complete data, and class factorised leaves). In contrast, the other methods run inference as if the model is a single PC. One can interpret that as giving different weights to each of the base models according to the likelihood of the instance to be classified (base models under which the instance is more likely are given higher weights). This inference method is referred to as GeF+ in the paper, as it defines a mixture over the base models.

`classify` and `classify_avg` will yield **different** class probabilities.

In [11]:
pred, prob = GeF.classify(X_test, return_prob=True)
prob

array([[0.385763  , 0.614237  ],
       [0.00223311, 0.99776689],
       [0.01383098, 0.98616902],
       [0.32482237, 0.67517763],
       [0.99795496, 0.00204504],
       [0.94311381, 0.05688619],
       [0.05254053, 0.94745947],
       [0.41572931, 0.58427069],
       [0.99843694, 0.00156306],
       [0.02609706, 0.97390294]])

In [12]:
pred, prob = GeF.classify_avg(X_test, return_prob=True)
prob

array([[0.29000003, 0.70999997],
       [0.10000002, 0.89999998],
       [0.10000008, 0.89999992],
       [0.36000006, 0.63999994],
       [0.74000001, 0.25999999],
       [0.74000002, 0.25999998],
       [0.32999996, 0.67000004],
       [0.42999992, 0.57000008],
       [0.78999997, 0.21000003],
       [0.23000002, 0.76999998]])

`classify` and `classify_lspn` (if both applicable) will yield **the same** class probabilities.

In [13]:
pred, prob = GeF.classify_lspn(X_test, return_prob=True)
prob

array([[0.385763  , 0.614237  ],
       [0.00223311, 0.99776689],
       [0.01383098, 0.98616902],
       [0.32482237, 0.67517763],
       [0.99795496, 0.00204504],
       [0.94311381, 0.05688619],
       [0.05254053, 0.94745947],
       [0.41572931, 0.58427069],
       [0.99843694, 0.00156306],
       [0.02609706, 0.97390294]])

Same for `classify_avg` and `classify_avg_lspn`.

In [14]:
pred, prob = GeF.classify_avg_lspn(X_test, return_prob=True)
prob

array([[0.29000003, 0.70999997],
       [0.10000002, 0.89999998],
       [0.10000008, 0.89999992],
       [0.36000006, 0.63999994],
       [0.74000001, 0.25999999],
       [0.74000002, 0.25999998],
       [0.32999996, 0.67000004],
       [0.42999992, 0.57000008],
       [0.78999997, 0.21000003],
       [0.23000002, 0.76999998]])